In [1]:
import string
import pandas as pd
import re
from unidecode import unidecode
from collections import Counter

In [2]:
#Importamos el Dataset que se ha scrapeado de la Web con todos los Lyrics
df = pd.read_csv('Lyrics.csv')
#Reordenamos las columnas
df = df[['id','Track','Artists','Lyric']]
#Les cambiamos el nombre
df.rename(columns = {'Lyric':'Lyrics'}, inplace = True)
#Nos cargamos las filas que no tengan canciones
df = df[['id','Track','Artists','Lyric']].dropna() 
#Nos cargamos las columnas que tengan featuring
df.drop(df[df['WordCount'].str.contains('ft', regex=False)].index,inplace=True) 
#Filas que no tienen canciones 318 166 1232 1238 1064 704 137, encontradas por busqueda manual
df.drop([318,166, 1232, 1238, 1064, 704, 137],inplace=True)

In [4]:
#Funcion para contar palabras de cada canción
def WordCounter(song_lyrics):
    raw_lyrics = song_lyrics.lower() #Letra de la cancion toda en minuscula
    raw_lyrics = re.sub(f"\[.*?\]", '' ,raw_lyrics) #Removemos de la letra todo lo que este entre [ ]
    raw_lyrics = unidecode(raw_lyrics) #Quitamos acentos
    raw_lyrics = raw_lyrics.translate(str.maketrans('', '', string.punctuation+'¿¡')) #Removemos la puntuación
    Lyric_list = re.split(r"[\u200b\s]+", raw_lyrics, flags=re.UNICODE) #Separamos por todos los tipos de espacio posibles

    return Counter(word for word in Lyric_list if word not in stop_words) #Retorna un BoW

In [5]:
#Creamos una lista de Stop Words
with open('stop_words_english.txt', encoding='utf-8') as f:
        stop_words_string = f.read()
        stop_words_string = stop_words_string.translate(str.maketrans('', '', "'-.")) #Removemos apostrofes
        english_stop_words_list = stop_words_string.split()
        
with open('stop_words_spanish.txt', encoding='utf-8') as f:
        stop_words_string = f.read()
        stop_words_string = unidecode(stop_words_string) #Quitamos acentos
        spanish_stop_words_list = stop_words_string.split()     
        
stop_words=['']
stop_words.extend(spanish_stop_words_list)
stop_words.extend(english_stop_words_list)

In [9]:
#Usamos assign para agregar una columna nueva para el WordCount
#Creamos una columna con el WordCount de cada Lyric
df = df.assign(WordCount=df.apply(lambda row: '' if pd.isnull(row['Lyrics']) else WordCounter(row['Lyrics']), axis=1).values )
#Guardamos el dataset en un csv
df.to_csv('Lyrics_WordCount.csv')

In [41]:
#Creamos un elemento Counter que englobe todas las canciones
DataSetWords = Counter()
for song_lyrics in df['Lyrics']:
    BagOfWords = WordCounter(song_lyrics)
    DataSetWords.update(BagOfWords)

In [42]:
#Guardamos el dataset en un csv
import csv #Convertimos el diccionario en un CSV
with open('DataSetWords.csv', 'w') as f:
    for key in DataSetWords.keys():
        f.write("%s,%s\n"%(key,DataSetWords[key]))